# QPIXL

In [56]:
# qip/helper.py

import numpy as np

def sfwht(a):
    n = len(a)
    k = ilog2(n)
    j = 1
    while j < n:
        for i in range(n):
            if i & j == 0:
                j1 = i + j
                x = a[i]
                y = a[j1]
                a[i], a[j1] = (x + y) / 2, (x - y) / 2
        j *= 2
    return a            

def isfwht(a):
    n = len(a)
    k = ilog2(n)
    j=1
    while j< n:
        for i in range(n):
            if (i&j) == 0:
                j1=i+j 
                x=a[i]
                y=a[j1]
                a[i],a[j1]=(x+y),(x-y)
        j*=2 
    return a            

def ispow2(x):
    return not (x&x-1)

def nextpow2(x):
    x-=1
    x|=x>>1
    x|=x>>2
    x|=x>>4
    x|=x>>8
    x|=x>>16
    x|=x>>32
    x+=1
    return x 

def ilog2(x):
    return int(np.log2(x))

def grayCode(x):
    return x^(x>>1)

def grayPermutation(a):
    b = np.zeros(len(a))
    for i in range(len(a)):
        b[i] = a[grayCode(i)]
    return b

def invGrayPermutation(a):
    b = np.zeros(len(a))
    for i in range(len(a)):
        b[grayCode(i)] = a[i]
    return b

def convertToAngles(a):
    scal = np.pi/(a.max()*2)
    a = a *scal
    return a

def convertToGrayscale(a,maxval=1):
    scal = 2*maxval/np.pi 
    a = a * scal
    return a

def countr_zero(n, n_bits=8):
    """Returns the number of consecutive 0 bits in the value of x, starting from the least significant bit ("right")."""
    counts = [0]
    count = 0
    while n != 0:
        if n & 1 == 0:
            count += 1
        elif count > 1:
            counts.append(count)
            count = 0
        n >>= 1
    return max(counts)

def countr_zero(n,n_bits=8):
    if n == 0:
        return n_bits
    count = 0
    while n & 1 == 0:
        count += 1
        n >>= 1
    return count


In [77]:
# qip/qpixl.py
from qiskit import QuantumCircuit


def cFRQI(a, compression):
    # a = convertToAngles(a, 1) # convert grayscale to angles
    
    n = len(a)
    k = ilog2(n)
    # print(a)

    a = 2*a 
    a = sfwht(a)
    a = grayPermutation(a) 
    a_sort_ind = np.argsort(np.abs(a))
    # print(a)

    # set smallest absolute values of a to zero according to compression param
    cutoff = int((compression / 100.0) * n)
    for it in a_sort_ind[:cutoff]:
        a[it] = 0
    # print(a)
    # Construct FRQI circuit
    circuit = QuantumCircuit(k + 1)
    # Hadamard register
    circuit.h(range(k))
    # Compressed uniformly controlled rotation register
    ctrl, pc, i = 0, 0, 0
    while i < (2**k):
        # Reset the parity check
        pc = int(0)

        # Add RY gate
        if a[i] != 0:
            circuit.ry(a[i], k)

        # Loop over sequence of consecutive zero angles
        if i == ((2**k) - 1):
            ctrl=0
        else:
            ctrl = grayCode(i) ^ grayCode(i+1)
            # print('ctrl gray', ctrl)
            ctrl = k - countr_zero(ctrl, n_bits=k+1) - 1
            # print('ctrl post op', ctrl)

        # Update parity check
        pc ^= (2**ctrl)
        # print('parity   ',pc)
        i += 1
        
        while i < (2**k) and a[i] == 0:
            # Compute control qubit
            if i == ((2**k) - 1):
                ctrl=0
            else:
                ctrl = grayCode(i) ^ grayCode(i+1)
                # print('ctrl gray', ctrl)
                ctrl = k - countr_zero(ctrl, n_bits=k+1) - 1
                # print('ctrl post op', ctrl)

            # Update parity check
            pc ^= (2**ctrl)
            # print('parity   ',pc)
            i += 1
            
        # print ('BREAK')
            
        for j in range(k):
            if (pc >> j)  &  1:
                # print('ctrl applied   ',j)
                circuit.cnot(j, k)
                
    return circuit

In [78]:
def readpgm(name):
    """Class to read pgm files as in original QPIXL, only for testing purposes"""
    with open(name) as f:
        lines = f.readlines()
    # This ignores commented lines
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    # here,it makes sure it is ASCII format (P2)
    assert lines[0].strip() == 'P2' 
    # Converts data to a list of integers
    data = []
    for line in lines[1:]:
        data.extend([int(c) for c in line.split()])
    return (np.array(data[3:]),(data[1],data[0]),data[2])

def preprocess_image(img):
    """Program requires flattened transpose of image array, this returns exactly that"""
    return img.T.flatten()


In [79]:
circuit = cFRQI(a, compression)

In [ ]:
circuit.draw()

In [ ]:
### EXAMPLE FOR DECODING - NOT CURRENTLY WORKING
import warnings 
warnings.filterwarnings("ignore")

import numpy as np
from typing import Dict, List

# Importing standard Qiskit libraries
import qiskit
from qiskit import QuantumCircuit, transpile, Aer, BasicAer, execute
from qiskit.algorithms.optimizers import COBYLA
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit_aer import StatevectorSimulator
from qiskit.providers.aer import QasmSimulator
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator

from matplotlib import pyplot as plt
import math
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import normalize
# from sklearn.metrics import mean_squared_error
from collections import Counter
from tensorflow.keras.datasets import fashion_mnist


(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
backend = StatevectorSimulator()
img = np.pad(X_train[0].flatten(),(0,240))
img = convertToAngles(img,255)
qc = cFRQI(img,0)
    
job = backend.run(qc)
sv = job.result().get_statevector()
state = np.array(np.abs(sv))

for i in range(0,2**qc.width(),2): ### FROM compressedFRQI.cpp file 
    state[int(i/2)]=np.arctan2(state[i+1],state[i])    
state = state[0:int(len(state)/2)]
state = convertToGrayscale(state,255)

im_shape = (28,28)
im_pix = im_shape[0]*im_shape[1]
plt.imshow(np.reshape(state[0:im_pix], im_shape))